<a href="https://colab.research.google.com/github/saqlain2204/GenAI-NLP-Resources/blob/main/LLAMA_3_Fine_tuning_%F0%9F%9B%A0%EF%B8%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Banner Image -->
<img src="https://i.ibb.co/NWMSWGF/Blue-Modern-Lets-Do-This-Linked-In-Banner-1.png" width="100%">

<!-- Links -->
<center>
  <a href="https://docs.inferless.com/" style="color: #B8FF33;">Docs</a> •
  <a href="https://tutorials.inferless.com/" style="color: #B8FF33;">Tutorials</a> •
  <a href="https://0ooatrmbp25.typeform.com/to/nzuhQtba?typeform-source=www.inferless.com" style="color: #06b6d4;">Join Private Beta</a>
</center>

# Finetune and Inference Llama 3 🛠️

Welcome to the Tutorial! 🚀

Let's dive into the supervised fine-tuning with Llama 3 8B model, released by Meta in April 2024. 📚 Llama 3 models were trained on 8x more data on over 15 trillion tokens. It has a context length of 8K tokens and increases the vocabulary size of the tokenizer to tokenizer to 128,256 (from 32K tokens in the previous version).

🌟 In this notebook we will finetune LLama 3 8B model, fine-tuning is the process of taking a pre-trained large language model (LLM) and further training it on a smaller, domain-specific dataset to improve its performance on specific tasks or in certain domains.

### Let's get started! 🌈







### Install the required *Libraries*

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

<h4>We will utilize Weights & Biases for metric tracking. Please provide your API key as input.</h4>
<h4>Additionally, ensure you include your Hugging Face token, as it is necessary when uploading the model to the Hugging Face Hub.</h4>


In [ ]:
#Login to wandb
import wandb
wandb.login()

model_name = "meta-llama/Meta-Llama-3-8B"
new_model = "inferless-Llama-3-8B"
#Copy and paste your hf_token
hf_token=""

### Import and load all the required libraries

In [ ]:
import os
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from trl import SFTTrainer,setup_chat_format

### Tokenizer and Model
Load and initialize the tokenizer with Hugging Face Transformers `AutoTokenizer`.

For `ChatML` support we will use `setup_chat_format()` function in `trl` . It will set up the `chat_template` of the tokenizer, adds special tokens to the `tokenizer` and  resizes the model’s embedding layer to accommodate the new tokens.

We will define the `BitsAndBytes` configurations and load the model in the 4-bit precision.

 Prepare the model for QLoRA training using the `prepare_model_for_kbit_training()`.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,token=hf_token)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0},token=hf_token)

model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

<a name="Data"></a>
### Format the dataset
Load and preprocess the `HuggingFaceH4/ultrachat_200k` dataset which is a filtered version of the UltraChat dataset from Huggingface.

Format the conversation using the `ChatML` template conversational style finetuning.

In [ ]:
dataset_name = "HuggingFaceH4/ultrachat_200k"
dataset = load_dataset(dataset_name, split="train_sft")
dataset = dataset.shuffle(seed=42).select(range(10000))

def format_chat_template(row):
    chat = tokenizer.apply_chat_template(row["messages"], tokenize=False)
    return {"text":chat}

processed_dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

dataset = processed_dataset.train_test_split(test_size=0.01)

### Training Configurations

Define the `LoRA Configuration` and the `Training Arguments` which will be used in the TRL's `SFTTrainer`. The SFTTrainer is then created and used to start the fine-tuning process.


In [ ]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

training_arguments = TrainingArguments(
        output_dir="./results_llama3_sft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=8e-6,
        eval_steps=10,
        # max_steps=None,
        num_train_epochs=1,
        warmup_steps=30,
        lr_scheduler_type="linear",
)

### 🚀 Start the Training Process

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=2024,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

### Save the final checkpoint

In [ ]:
trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

#load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)


#Merge the base model with the "final_checkpoint" adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

#Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

#Push them to the HF Hub
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)